# Hackacity 2024
## [SOLUTION NAME]
#### Team: PLICity
#### Authors:
- José Cabeda
- Lirielly 
- Pedro Machado
- Igor Carvalho

# Instructions
- The purpose of this notebook is to provide a detailed description so that the technical and business jury can have an overview of the technical and non-technical aspects and how they interconnect.
- To use the template, you must make a copy and fill in the team name and the challenge title.
- The cells containing the instructions should be deleted, but all headers must be kept.
- All technical information should be included in this report (e.g., code, queries, graphs, tools used, parameters, etc.). If necessary, additional files can be added to the OneDrive folder, with references to those files included in this report.

# Tip
- The technical and business jury will need to evaluate several notebooks, so it’s important to maintain a clear line of thought within the notebook without overloading it with too many similar graphs or visuals that do not add value to the work. On the other hand, it’s important to show that the work done is thorough.
- Therefore, we recommend using the annex section to include code that was produced along with its description but might be unnecessary noise for the jury (e.g., tested models that didn’t work, EDA with very similar results between variables, etc.).


# 1 - Introduction


The wifi acess data provides us with a way to track the mobility of the citizens. According to the paper A Survey on Privacy in Human Mobility "human mobility data can be used for different scopes such as urban traffic management, urban planning, urban pollution estimation, etc". With the rise of AI many use cases devolve around having a good dataset without biases and with relevant data.
In this work we:

- Propose the usage of this dataset to predict the access points where the number of sessions surpasses a given threshold
- Propose how to generate a synthetic dataset for these types of uses cases
- Prove that this synthetic data can  be used for our model

Furthermore we provide an analysis that the dataset, even if anonymised, could be used for individual tracking by reverse engineering the location of the datasets based on the number of sessions in a given day, by matching with main events in the city of Porto. The same shouldn't occur with the synthetized dataset.

# 2 - Possible threat analysis before anonymization

### Instructions:
This section should cover a threat analysis of the dataset from the privacy perspective and it's quantification.

Important notes:
- Only this section is evaluated for the data privacy award;
- No code is needed for this section;
- A good quantification of the threat is valuade by the jury;
- Include any references that is important.

The challenge provides us with two separate datasets. The access points and the wifi_locations. Focusing on the wifi access dataset we find the following threats.

1. Detailed Movement Tracking
A possible issue with the wifi access without anonymization is that we can hypotheticaly track a singular user. An attacker could piece together a user's daily routine by:
  - Tracking consistent MAC addresses on the wifi sessions
  - Analyzing connection timestamps (`acctstarttime` and `acctsessiontime`)
  - Mapping access point locations by correlating with the `callingstationid`

For example consider the following
  ```
  User consistently connects to:
  - 7:30 AM: Access Point near home
  - 8:00 AM: Access Point near workplace
  - 12:30 PM: Access Point in lunch area
  - 6:00 PM: Access Point near home again
  ```

- This pattern reveals:
  - Home location
  - Work location
  - Daily routine
  - Potentially sensitive personal information

2. Data Correlation Risks

With the dataset and the identification of the dataset, it's possible for other entities to cross-reference:
  - with public social media
  - Location-based services (i.e. strava)
  - Other open datasets

With this dataset it's possible to:
  - target advertisments (given a mac address present ads on restaurants or shops it usually goes by)
  - allow potential stalking
  - enable social engineering attacks (i.e. answer questions like what is the your favorite restaurant?)

# Risk Assessment Scorecard

There are 3 possible values (none -0 | low -1  | high - 2). On top of the risks above we define the following categories to help track the impact of having the dataset available:

- Data Breach Impact
- Legal and Regulatory Compliance
- Technical Safeguards
- Monitoring and Auditing
- User Education and Awareness
- Incident Response Plan

# Risk Assessment Scorecard

| Risk Category                   | Status |
|-----------------------------    |--------|
| Detailed Movement Tracking      | 2      |
| Data Correlation Risks          | 2      |
| Data Breach Impact              | 2      |
| Legal and Regulatory Compliance | 2      |
| Technical Safeguards            | 2      |
| Monitoring and Auditing         | 2      |
| User Education and Awareness    | 2      |
| Incident Response Plan          | 2      |

Risk score: 16/16

## Mitigation Strategies

Considering the threats above we propose the following mitigation strategies:

1. **Data Minimization**:
- Reduce detailed location information (i.e. access points coordinates are approxiamted)
- Generalize timestamps (i.e. remove minutes or change date)

2. **Advanced Anonymization**:
   - Hashing MAC addresses (`callingstationid`)
   - Create random session IDs (`acctsessionid`)

These techniques allow to remove the direct connection between the access points and the wifi accesses datasets.

3. **Consent and Transparency**:
   - Clear user consent for data collection
   - Opt-out mechanisms
   - Detailed privacy policy

Although it doesn't mitigate the risks of approved data this enables the dataset to be given only by consenting users and in this way follow the GDPR which states that we can only store data a given user consents on.

4. **Technical Safeguards**:
   - Implement encryption for data at rest and in transit
   - Use secure access controls and authentication mechanisms

5. **Monitoring and Auditing**:
   - Regularly monitor access to the data and audit logs
   - Implement intrusion detection systems

7. **Incident Response Plan**:
   - Develop and maintain an incident response plan
   - Include steps for notifying affected individuals and regulatory authorities



# 3 - Anonymization process

### Instructions

Based on the previous threat assesment, propose any anonymization techniques you might find helpful.

Include all the code and proof needed that the threat has been mitigated.

## Synthetic data for ML
Considering the techniques which have already been applied to the dataset provided to us we have an hypothesis that we can potentially reverse engineer the relation between the access points by correlating across different datasets and link to real identities through additional context. 

For example in this work we propose the usage of datasets with the Porto city events and the university of Porto events and correlate them with access outliers. (i.e there's an access point in Inesc near FEUP, and given a couple of events at this place, if they closely correlate to the same dates of outliers we can say with confidence that the anonymised callingstationid refers to the given access point).

Some existing event datasets are:
- Cultural calendar of porto: https://opendata.porto.digital/dataset/8f4316c7-aafe-488f-a8b0-5e721ae37637/resource/351[…]95011584?activity_id=be273578-39f4-4c76-ae1d-db515d9f7362
- university of porto calendar: https://sigarra.up.pt/up/pt/web_base.gera_pagina?p_pagina=calend%c3%a1rio%20escolar%20da%20universidade%20do%20porto


# 4 - Possible threat analysis after anonymization

### Instructions

Not all solutions are perfect - Do a threat analysis and compare the threats before and after your solution.

After the anonymization of the data we can consider 3 different stages:

1. Data is anonymized
2. Processes are implemented
3. Synthetic dataset is provided 

Given these stages we have the following score cards:

# Data is anonymized

Score: 14/16

| Risk Category                   | Status |
|-----------------------------    |--------|
| Detailed Movement Tracking      | 1      |
| Data Correlation Risks          | 1      |
| Data Breach Impact              | 2      |
| Legal and Regulatory Compliance | 2      |
| Technical Safeguards            | 2      |
| Monitoring and Auditing         | 2      |
| User Education and Awareness    | 2      |
| Incident Response Plan          | 2      |

# 2 - Processes are implemented

Score: 8/16

| Risk Category                   | Status |
|-----------------------------    |--------|
| Detailed Movement Tracking      | 1      |
| Data Correlation Risks          | 1      |
| Data Breach Impact              | 1      |
| Legal and Regulatory Compliance | 1      |
| Technical Safeguards            | 1      |
| Monitoring and Auditing         | 1      |
| User Education and Awareness    | 1      |
| Incident Response Plan          | 1      |

# 3 - Synthezized data is provided

Score: 4/16

| Risk Category                   | Status |
|-----------------------------    |--------|
| Detailed Movement Tracking      | 0      |
| Data Correlation Risks          | 0      |
| Data Breach Impact              | 0      |
| Legal and Regulatory Compliance | 1      |
| Technical Safeguards            | 1      |
| Monitoring and Auditing         | 1      |
| User Education and Awareness    | 1      |
| Incident Response Plan          | 1      |

# 5 - Data potential and analysis

### Instructions

It is typical that during an anonymization process, there is a loss of value in terms of insights. In this section, explore the other side of the scale - What can still be done with the data that is processed and why is the risk above justifiable taking into account the potential return?

Specifically, try to imagine yourself as the city's analytics team, which uses this data to gather conclusions about the city's state and flow and try develop and demonstrate what results and analyses can be done. You can use the given data plus optionally any other data sources. If you complement with additional datasets, be it from the city Open Data Portal (https://opendata.porto.digital) or other datasets, make sure you cite and reference them explicitly.



In [ ]:
# Given the dataset predict the access points that will have the most demand
# 1.1 Com o dataset original (can tell the exact point to dynamically allocate resources)
# 1.2 Run with the synthetic data and check the results are good

# 5 - Conclusions and Future Work


### Instructions:

List the main conclusions focusing on the feasibility, innovation, and applicability of the solution.

In addition, describe the future work still necessary if this solution were to continue: What would the next steps be? What limitations do the data have for implementing the solution? How can the data be improved? How could this solution be improved? What other ideas could be included/analyzed? What other types of data could be used? What other methodologies could be experimented with?

# 6 - References

### Instructions:

Provide the relevant references for materials and/or sources used (reports, articles, external data sources, etc.).


# 9 - Annex